# Chronos-2 Multivariate vs Univariate Forecasting

This notebook compares UV and MV forecasting using Chronos-2 on financial time series.

## Research Questions

1. Do multivariate methods produce better predictions than univariate when using foundation models?
2. Is MV forecasting accuracy better for stocks versus interest rates?
3. Is MV forecasting better when both stocks and interest rates are forecast together?
4. Can we build a large-scale "world" forecasting model?

## Setup

In [ ]:
import pandas as pd
import torch
from data_fetcher import DataFetcher
from chronos_forecaster import ChronosForecaster
from metrics import compare_uv_mv_metrics, print_metrics_comparison
from visualizer import plot_forecast_comparison, plot_error_comparison, plot_metrics_summary
from experiment_runner import ExperimentRunner
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Quick Test - Single Experiment

Let's start with a simple test to make sure everything works.

In [ ]:
# Fetch Magnificent-7 stocks
fetcher = DataFetcher()
stocks_df = fetcher.fetch_stocks(start_date="2023-01-01", end_date="2024-12-31")
print(f"\nFetched data shape: {stocks_df.shape}")
stocks_df.head()

In [ ]:
# Prepare data for Chronos-2
stocks_df["item_id"] = "stocks"

# Split into context and test
context_df = stocks_df.iloc[:400].copy()
test_df = stocks_df.iloc[400:445].copy()

print(f"Context: {len(context_df)} days")
print(f"Test: {len(test_df)} days")

In [ ]:
# Initialize forecaster with base model (recommended for accuracy/speed balance)
# Options: model_size="small", "base", or "large"
device = "cuda" if torch.cuda.is_available() else "cpu"
forecaster = ChronosForecaster(device=device, model_size="base")

print("\nNote: Using 'base' model. For higher accuracy (but slower), use model_size='large'")

In [ ]:
# Run UV vs MV comparison for NVDA
target = "NVDA"
results = forecaster.compare_uv_mv(
    context_df, 
    test_df, 
    target, 
    prediction_length=len(test_df)
)

In [ ]:
# Calculate and print metrics
comparison = compare_uv_mv_metrics(results)
print_metrics_comparison(comparison)

In [ ]:
# Visualize forecasts
fig = plot_forecast_comparison(results, title=f"{target} Stock Price Forecast")
plt.show()

In [ ]:
# Plot errors
fig = plot_error_comparison(results)
plt.show()

## 2. Experiment 1: Magnificent-7 Stocks

Test all 7 stocks with UV vs MV forecasting.

In [ ]:
# Run for all stocks
all_comparisons = []

for target in ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA"]:
    print(f"\n{'='*60}")
    print(f"Processing {target}...")
    print(f"{'='*60}")
    
    results = forecaster.compare_uv_mv(
        context_df, 
        test_df, 
        target, 
        prediction_length=len(test_df)
    )
    
    comparison = compare_uv_mv_metrics(results)
    all_comparisons.append(comparison)
    print_metrics_comparison(comparison)

In [ ]:
# Summary visualization
fig = plot_metrics_summary(all_comparisons)
plt.show()

## 3. Experiment 2: Interest Rates

Test FRED interest rates with UV vs MV forecasting.

In [ ]:
# Fetch interest rates
rates_df = fetcher.fetch_interest_rates(start_date="2023-01-01", end_date="2024-12-31")
print(f"\nFetched rates shape: {rates_df.shape}")
rates_df.head()

In [ ]:
# Prepare and split
rates_df["item_id"] = "rates"
context_rates = rates_df.iloc[:400].copy()
test_rates = rates_df.iloc[400:445].copy()

print(f"Context: {len(context_rates)} days")
print(f"Test: {len(test_rates)} days")

In [ ]:
# Run for key maturities
rate_comparisons = []

for target in ["DGS3MO", "DGS1", "DGS5", "DGS10", "DGS30"]:
    print(f"\n{'='*60}")
    print(f"Processing {target}...")
    print(f"{'='*60}")
    
    results = forecaster.compare_uv_mv(
        context_rates, 
        test_rates, 
        target, 
        prediction_length=len(test_rates)
    )
    
    comparison = compare_uv_mv_metrics(results)
    rate_comparisons.append(comparison)
    print_metrics_comparison(comparison)

In [ ]:
# Summary visualization
fig = plot_metrics_summary(rate_comparisons)
plt.show()

## 4. Experiment 3: Combined (Stocks + Rates)

Test with both stocks and interest rates together.

In [ ]:
# Fetch combined data
combined_df = fetcher.fetch_combined(start_date="2023-01-01", end_date="2024-12-31")
print(f"\nCombined data shape: {combined_df.shape}")
combined_df.head()

In [ ]:
# Prepare and split
combined_df["item_id"] = "combined"
context_combined = combined_df.iloc[:400].copy()
test_combined = combined_df.iloc[400:445].copy()

print(f"Context: {len(context_combined)} days")
print(f"Test: {len(test_combined)} days")
print(f"Total series: {len(combined_df.columns) - 2}")  # Exclude timestamp and item_id

In [ ]:
# Test a few key series from combined dataset
combined_comparisons = []

for target in ["AAPL", "NVDA", "DGS10", "DGS30"]:
    print(f"\n{'='*60}")
    print(f"Processing {target} with combined dataset...")
    print(f"{'='*60}")
    
    results = forecaster.compare_uv_mv(
        context_combined, 
        test_combined, 
        target, 
        prediction_length=len(test_combined)
    )
    
    comparison = compare_uv_mv_metrics(results)
    combined_comparisons.append(comparison)
    print_metrics_comparison(comparison)

## 5. Full Parameter Sweep (Optional)

Run comprehensive experiments across all parameter combinations.
Warning: This will take a long time!

In [ ]:
# Initialize experiment runner
runner = ExperimentRunner(device=device)

# Fetch full historical data
full_stocks = fetcher.fetch_stocks(start_date="2020-01-01", end_date="2024-12-31")
full_stocks["item_id"] = "stocks"

In [ ]:
# Run parameter sweep for stocks
# Note: Start with a smaller date range for testing
stock_results = runner.run_parameter_sweep(
    df=full_stocks,
    target_columns=["AAPL", "NVDA"],  # Start with 2 stocks
    history_multipliers=[1, 2],  # Test 1 and 2 years
    forecast_horizons=[21],  # Just 1 month
    start_date="2024-01-01",
    end_date="2024-06-30",
    step_months=3  # Quarterly
)

In [ ]:
# Get summary statistics
summary = runner.get_summary_stats()

In [ ]:
# Save results
runner.save_results("stock_experiment_results.json")

## 6. Analysis and Conclusions

Answer the research questions based on experimental results.

In [ ]:
# Compare stocks vs rates
print("\n" + "="*60)
print("STOCKS vs RATES COMPARISON")
print("="*60)

stocks_mv_win_rate = sum(c["mv_better"] for c in all_comparisons) / len(all_comparisons) * 100
rates_mv_win_rate = sum(c["mv_better"] for c in rate_comparisons) / len(rate_comparisons) * 100

print(f"\nStocks MV Win Rate: {stocks_mv_win_rate:.1f}%")
print(f"Rates MV Win Rate: {rates_mv_win_rate:.1f}%")

stocks_avg_improvement = sum(c["mape_improvement"] for c in all_comparisons) / len(all_comparisons)
rates_avg_improvement = sum(c["mape_improvement"] for c in rate_comparisons) / len(rate_comparisons)

print(f"\nStocks Avg MAPE Improvement: {stocks_avg_improvement:.2f}%")
print(f"Rates Avg MAPE Improvement: {rates_avg_improvement:.2f}%")

## Conclusions

Based on the experiments above, document your findings for each research question:

1. **UV vs MV with Foundation Models:** [Your findings here]
2. **Stocks vs Interest Rates:** [Your findings here]
3. **Joint Forecasting:** [Your findings here]
4. **World Model Feasibility:** [Your findings here]